In [1]:
import requests
import json
from tabulate import tabulate

In [2]:
def get_access_token(user='abbatelucas@gmail.com', password='Futbol !0') -> str:
    headers = {
        "authority":"api.cocos.capital",
        "apikey":"eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyAgCiAgICAicm9sZSI6ICJhbm9uIiwKICAgICJhdWRpZW5jZSI6ICJjb2NvcyIsCiAgICAiaXNzIjogInN1cGFiYXNlIiwKICAgICJpYXQiOiAxNjQxOTU2NDAwLAogICAgImV4cCI6IDM5NDgzNDE1MzEKfQ.Q5ZiL7KCUKP7iSM_LHWd3gffZ0k5Ce6CemOX9CUfEdM",
        "origin":"https://app.cocos.capital",
        "referer":"https://app.cocos.capital/",
        "sec-ch-ua-mobile":"?0",
        "sec-ch-ua-platform":"Windows",
        "sec-fetch-dest":"empty",
        "sec-fetch-mode":"cors",
        "sec-fetch-site":"same-site",
        }
    data = json.dumps({'email': user, 'password': password})
    r = requests.post('https://api.cocos.capital/auth/v1/token?grant_type=password', headers=headers, data=data)
    s = r.content.decode('utf-8')
    return json.loads(s)['access_token']

# Pasar autentificacion como archivo


In [3]:
token = get_access_token()

In [24]:
def get_ticker_data(ticker: str, currency: str = 'auto', token: str = get_access_token(), plazo = 48):
    plazo_conversion = {'48': '0003',
                        '24': '0002', 
                        'CI': '0001',
                        'C.I.': '0001'}
    liquidacion_str = plazo_conversion[str(plazo).upper()]
    
    if currency == 'auto': # Si currency es auto, pasa a usd si termina en D o C (para casos como AMD, no va a andar auto, hay que poner currency ARS)
        if ticker[-1] in ['D', 'C']:
            currency = 'USD'
            ticker = ticker[:-1]
        else:
            currency = 'ARS'
            
        
    currency = 'USD' if currency == 'D' else currency
    species = 'D' if currency == 'USD' else ''
    
    
    headers = {
        "accept": "application/json, text/plain, */*",
        "accept-language": "es-ES,es;q=0.9",
        "authorization": f"Bearer {token}",
        "if-none-match": "W/\"42-WJmuy/h7CrOCewTOtVPh5zV6UTY\"",
        "recaptcha-token": "undefined",
        "sec-ch-ua": "\"Chromium\";v=\"118\", \"Google Chrome\";v=\"118\", \"Not=A?Brand\";v=\"99\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-site",
        "x-account-id": "54107"
    }
    
    link = f"https://api.cocos.capital/api/v1/markets/ticker/{ticker}{species}-{liquidacion_str}-C-CT-{currency}"
    r = requests.get(link, headers=headers)
    if r.status_code == 200:
        return r.json()
    else:
        print(r.content)
        raise Exception('Error en la consulta')
    
    
def gen_table(ticker: str, currency: str = 'auto', token: str = get_access_token(), plazo = 48):
    data = get_ticker_data(ticker, currency, token, plazo)
    
    compra_size = []
    compra_price = []
    for row in data['bids']:
        compra_size.append(row['size'])
        compra_price.append(row['price'])
        
    venta_size = []
    venta_price = []
    for row in data['asks']:
        venta_size.append(row['size'])
        venta_price.append(row['price'])

    
    table = {'Cant. compra': compra_size, 'Precio compra': compra_price, '': '', 'Precio venta': venta_price, 'Cant. venta': venta_size}
    
    print('\t', data['short_ticker'], '-', data['instrument_name'], '-', data['currency'])
    print('\t', f'Last: {data["last"]:,} - High: {data["high"]:,} - Low: {data["low"]:,} - Volume: {data["volume"]:,}')

    print(tabulate(table, headers='keys', tablefmt='rounded_outline', floatfmt='.2f', stralign='center'))

In [63]:
import requests
import json
import codecs
import sys, os
from tabulate import tabulate
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
from cryptography.fernet import Fernet

def encrypt(data, key):
    fernet = Fernet(key)
    return fernet.encrypt(data)

def decrypt(data, key):
    fernet = Fernet(key)
    return fernet.decrypt(data)

def gen_credentials(user, password, key=b'DJ4hV8Bzq_UVRqtKvHbVwwlr9zJDFAhVxro0S3tE4QM=', path="coki_credentials.txt"):
    with open(path, "wb") as f:
        user = encrypt(str(user).encode(), key)
        password = encrypt(str(password).encode(), key)
        f.writelines([user, b'\n', password])

In [62]:
gen_credentials('abbatelucas@gmail.com', 'Futbol !0')

In [68]:
def read_credentials(key=b'DJ4hV8Bzq_UVRqtKvHbVwwlr9zJDFAhVxro0S3tE4QM=', path="coki_credentials.txt") -> (str, str):
    with open(path, "r", encoding='utf8') as f:
        user, password = f.readlines()
        user = user.strip()

    return decrypt(user, key).decode(), decrypt(password, key).decode()


In [69]:
read_credentials()

('abbatelucas@gmail.com', 'Futbol !0')

In [25]:
gen_table('QQQD')

	 QQQD - Cedear Invesco QQQ Trust ETF - USD
	 Last: 21.2 - High: 21.4 - Low: 19.3 - Volume: 1,401
╭────────────────┬─────────────────┬────┬────────────────┬───────────────╮
│   Cant. compra │   Precio compra │    │   Precio venta │   Cant. venta │
├────────────────┼─────────────────┼────┼────────────────┼───────────────┤
│             50 │           19.00 │    │          21.50 │            95 │
│              3 │           16.00 │    │                │               │
│              5 │           15.40 │    │                │               │
╰────────────────┴─────────────────┴────┴────────────────┴───────────────╯
